In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("downloads/65k_enriched_topic.csv")

In [332]:
df["len"] = df.text.apply(lambda t: len(t.split(" ")))

In [333]:
df = df[df["len"] > 7]

In [334]:
pcm = pd.read_csv("downloads/PMC-ids.csv")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [335]:
pcm.head()

,Journal Title,ISSN,eISSN,Year,Volume,Issue,Page,DOI,PMCID,PMID,Manuscript Id,Release Date
0,Breast Cancer Res,1465-5411,1465-542X,2000,3,1,55,10.1186/bcr271,PMC13900,11250746.0,NaN,live
1,Breast Cancer Res,1465-5411,1465-542X,2000,3,1,61,10.1186/bcr272,PMC13901,11250747.0,NaN,live
2,Breast Cancer Res,1465-5411,1465-542X,2000,3,1,66,10.1186/bcr273,PMC13902,11250748.0,NaN,live
3,Breast Cancer Res,1465-5411,1465-542X,1999,2,1,59,10.1186/bcr29,PMC13911,11056684.0,NaN,live
4,Breast Cancer Res,1465-5411,1465-542X,1999,2,1,64,10.1186/bcr30,PMC13912,11400682.0,NaN,live


In [346]:
import math
def lookup_DOI(val, lookup, key="PMCID"): 
#     print(val, type(val))
#     if val is None or math.isnan(val):
#         return None
#     print(val)
    if type(val) == str:
        #print(val)
        #print(val, type(val))

        match = lookup[key] == val
        DOI = lookup['DOI'][match]
        return DOI.values[0]
    return None

df["pdoi"] = df['PMCID'].apply(lambda x: lookup_DOI(x, pcm, "PMCID"))

In [ ]:
df[df["pdoi"].notna()]

,level_0,problem_id,title,article_link,text,mesh_list,journal_title,volume,issue,label,...,cluster_id,topic_id,probs,topic_name,closest_problem_id,doi,coreId,oai,len,pdoi
50,3295.0,2869.0,NaN,NaN,Parkinson's disease (PD) is an important disab...,NaN,NaN,NaN,NaN,1.0,...,2720.0,26,"[26, 108, 27]",parkinson dopaminergic neurodegenerative,"[2817, 2909, 2919]",NaN,NaN,NaN,17,10.3892/ijmm.2020.4806
63,1685.0,1486.0,NaN,NaN,"Cognitive impairment, including dementia, is a...",NaN,NaN,NaN,NaN,1.0,...,41.0,354,"[354, 364, 363]",dementia alzheimer caregivers,"[3250, 3168, 17200]",NaN,NaN,NaN,14,10.1186/1471-2458-14-643
126,3989.0,3403.0,NaN,NaN,The prevention of falls among older people is ...,NaN,NaN,NaN,NaN,1.0,...,367.0,8,"[8, 121, 199]",falls fall falling,"[22715, 21764, 8303]",NaN,NaN,NaN,13,10.1371/journal.pmed.1002112
165,179.0,178.0,NaN,NaN,Lung cancer is one of the most common human ma...,NaN,NaN,NaN,NaN,1.0,...,603.0,344,"[344, 343, 216]",lung radon death,"[1160, 242, 385]",NaN,NaN,NaN,19,10.1371/journal.pone.0128201
177,619.0,601.0,NaN,NaN,Depression and obesity are significant global ...,NaN,NaN,NaN,NaN,1.0,...,1625.0,-1,"[202, 201, 183]",depression depressive psychiatric,"[523, 462, 680]",NaN,NaN,NaN,12,10.1186/s40359-020-00400-w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66169,3674.0,3174.0,NaN,NaN,Hypertension in the elderly with cognitive imp...,NaN,NaN,NaN,NaN,1.0,...,1693.0,149,"[149, 148, 147]",cognitive memory fearful,"[3251, 1593, 14335]",NaN,NaN,NaN,15,10.1097/MD.0000000000020789
66289,3290.0,2864.0,NaN,NaN,A cardinal manifestation of chronic autonomic ...,NaN,NaN,NaN,NaN,1.0,...,1931.0,108,"[108, 397, 374]",hypertension cardiovascular orthostatic,"[19649, 23525, 17595]",NaN,NaN,NaN,23,10.1007/s10286-012-0176-4
66514,3990.0,3404.0,NaN,NaN,Gulf War illness (GWI) is a disorder related t...,NaN,NaN,NaN,NaN,1.0,...,2575.0,213,"[217, 213, 378]",guillain postinfectious hypokalaemic,"[1881, 14315, 7754]",NaN,NaN,NaN,15,10.1371/journal.pone.0177121
66526,2752.0,2403.0,NaN,NaN,Skin cancers remain the most common group of c...,NaN,NaN,NaN,NaN,1.0,...,742.0,293,"[293, 296, 300]",melanoma melasma cancers,"[3249, 22718, 16505]",NaN,NaN,NaN,16,10.1007/s40259-020-00428-9


In [108]:
len(pcm)

7828449

In [ ]:
df[["text", "topic_id"]]

,text,topic_id
0,Dengue is a potentially serious common viral i...,22
1,The article also includes new emerging zoonoti...,91
2,Chagas' disease is a potentially life-threaten...,40
3,Glaucoma is the third-leading cause of blindne...,348
4,Pancreatic cancer (PC) remains difficult to tr...,34
...,...,...
66542,The biggest problem with most biological monit...,-1
66543,Uterine myometrial tumors are the most commonl...,173
66544,The death of a young patient is a difficult bu...,414
66545,A major challenge in the post-genome era of pl...,163


In [6]:
topics = list(df["topic_name"].unique())#unique_values()

In [7]:
# topics #1200

In [352]:
with open("vocab30k.txt") as f:
    vocab30k = f.read().split("\t\n")

In [353]:
dfa = df[:15]

In [354]:
def extract_domain_words(sent, vocab=vocab30k):
    words = sent.split(" ")
    domain = []
    for word in words:
        if word not in vocab:
            domain += [word]
    return domain

In [355]:
df["domain_words"] = df["text"].apply(extract_domain_words)

 ## TODOs:
 
 * find frequency of domain words within cluster
 * find subfrequency of that word (coocurrence)

# Keyphrase extraction

In [357]:
!pip install keybert


In [360]:
from keybert import KeyBERT
kw_model = KeyBERT()


In [361]:
keywords

[('supervised', 0.6676),
 ('labeled', 0.4896),
 ('learning', 0.4813),
 ('training', 0.4134),
 ('labels', 0.3947)]

In [362]:
kws = kw_model.extract_keywords("everybody has cervical cancer", keyphrase_ngram_range=(1, 2), stop_words='english', 
                              use_mmr=True, diversity=0.3)

kws.sort(key=lambda x: x[1])
kws

[('everybody', 0.3188),
 ('cancer', 0.5017),
 ('cervical', 0.5567),
 ('everybody cervical', 0.7245),
 ('cervical cancer', 0.7957)]

In [363]:
#%%timeit


In [364]:
sentences = list(df["text"])
len(sentences)

56895

In [365]:
from sentence_transformers import SentenceTransformer, util
import time

print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/889 [00:00<?, ?it/s]

In [366]:
len(embeddings)

56895

In [367]:
print("Start clustering")
start_time = time.time()

clusters = util.community_detection(embeddings, threshold=0.9)
print("Clustering took {:.2f} sec".format(time.time() - start_time), "len", len(clusters))

Start clustering
Clustering took 6.68 sec len 270


In [368]:
def get_kw(text):
    kws = kw_model.extract_keywords(text, 
                          keyphrase_ngram_range=(1, 2), 
                          stop_words='english',
                          use_mmr=True, 
                          diversity=0.2)
    
    kws.sort(key=lambda x: x[1], reverse=True)
    return kws

In [369]:
df["kws"] = df["text"].apply(get_kw)

In [370]:
def likely(ts):
    #if a unigram is in the two highest likely probs THEN take the unigram]
    try:
        s0=ts[0][0]
        s1=ts[1][0]
        if len(s0.split(" ")) > 1 and len(s1.split(" ")) == 1:
            return s1
        return s0
    except:
        return None


In [371]:
def contender(topics):
    #take the first "different" option that breaks with the main options
    #iterate until the topic that breaks with the terms of previous topics
    terms = []
    idx=0
    for idx, t in enumerate(topics):
        words = t[0].split(" ")
#         print(words, "words")
        if set.intersection(set(words), set(terms)):
            break #word
        terms += words
    return likely(topics[idx+1:])

In [372]:
df["kw1"] = df.kws.apply(likely)

In [374]:
df["kw2"] = df.kws.apply(contender)

In [375]:
df["kw1"]

0                   dengue
1        zoonotic bacteria
2           chagas disease
3                 glaucoma
4               pancreatic
               ...        
66542           biological
66543    myometrial tumors
66544                death
66545         plant genome
66546         diseases lsd
Name: kw1, Length: 56895, dtype: object

In [376]:
df["kw2"]

0                   viral infection
1        staphylococcus intermedius
2                       trypanosoma
3                         blindness
4              anticancer therapies
                    ...            
66542            problem biological
66543                neoplasm women
66544                   death young
66545                 plant biology
66546                     lysosomal
Name: kw2, Length: 56895, dtype: object

In [377]:
df.to_csv ("downloads/58k_enriched_topics.csv", index = False, header=True)

## Topic ANalysis

* check KW1 
* check sub KW2s for that KW

* group sentences (group their topics)
* build dictionaries of topics

In [3]:
df = pd.read_csv("downloads/58k_enriched_topics.csv")

In [11]:
len(set(list(df.kw1))), len(set(list(df.kw2)))

(30172, 28607)

In [7]:
len(set.intersection(set(list(df.kw1)), set(list(df.kw2))))

4500

In [212]:
for cluster_idxs in clusters[:1]:
    sample = df.iloc[cluster_idxs]
    sample

32541

In [217]:
df.iloc[[32541, 22]]

,level_0,problem_id,title,article_link,text,mesh_list,journal_title,volume,issue,label,...,topic_id,probs,topic_name,closest_problem_id,doi,coreId,oai,len,domain_words,pdoi
38007,18583.0,20735.0,NaN,NaN,Obesity represents a serious health problem as...,NaN,NaN,NaN,NaN,1.0,...,130,"[130, 131, 111]",obesity overweight epidemic,"[13282, 12611, 24022]",None,NaN,NaN,13,"[Obesity, .]",None
22,17266.0,19449.0,NaN,NaN,"Calcification , the major cause of bioprosthet...",NaN,NaN,NaN,NaN,1.0,...,394,"[394, 392, 393]",ventricular valves aortic,"[22850, 17528, 7007]",None,NaN,NaN,20,"[Calcification, ,, bioprosthetic, ,, .]",None


In [214]:
#Print for all clusters the top 3 and bottom 3 elements
clus = {}
for i, cluster in enumerate(clusters[:20]):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    clus["cluster_number " + str(i)] = cluster
    
    for sentence_id in cluster[0:3]:
        print("\t", sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", sentences[sentence_id])


Cluster 1, #241 Elements 
	 Obesity represents a serious health problem as it is rapidly increasing worldwide .
	 Obesity is a serious and rapidly growing health problem worldwide .
	 Obesity is a serious public health problem that is growing alarmingly worldwide .
	 ...
	 Morbid obesity and its rapidly increasing prevalence has became a serious health , social and economic problem not only in Europe , but in other developed countries in the Western world .
	 Obesity is a serious health problem for all age groups and it also promotes Type 2 diabetes , cardiovascular diseases , cancer and hypertension .
	 Obesity is a serious health problem that increased risk for many complications , including diabetes and cardiovascular disease .

Cluster 2, #118 Elements 
	 Hepatitis C virus ( HCV ) is a serious global health problem and establishes chronic infection in a significant number of infected humans worldwide .
	 Hepatitis C virus ( HCV ) is a serious global health problem affecting approx

# NetworkX hierarchy